# 30VNFoods Classification with Custom Trainer

30종류의 베트남 음식 중 10개의 카테고리를 선별해서 실습

## Data Load

In [1]:
import os
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense

In [2]:
# training set과 test set의 모든 이미지 파일에 대해서,
# jpg image header가 포함되지 않은 (jpg의 파일 구조에 어긋나는) 파일들을 삭제

data_path = '/aiffel/aiffel/model-fit/data/30vnfoods/'
train_path = data_path + 'Train/'
test_path = data_path + 'Test/'

for path in [train_path, test_path]:
    classes = os.listdir(path)

    for food in classes:
        food_path = os.path.join(path, food)
        images = os.listdir(food_path)
        
        for image in images:
            with open(os.path.join(food_path, image), 'rb') as f:
                bytes = f.read()
            if bytes[:3] != b'\xff\xd8\xff':
                print(os.path.join(food_path, image))
                os.remove(os.path.join(food_path, image))

In [3]:
classes = os.listdir(train_path)
train_length = 0

for food in classes:
    food_path = os.path.join(train_path, food)
    images = os.listdir(food_path)
    
    train_length += len(images)

print('training data의 개수: '+str(train_length))

training data의 개수: 9775


## Define Data Loader

In [4]:
import pathlib
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


def process_path(file_path, class_names, img_shape=(224, 224)):
    '''
    file_path로부터 class label을 만들고, 이미지를 읽는 함수
    '''
    label = tf.strings.split(file_path, os.path.sep)
    label = label[-2] == class_names

    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    img = tf.image.resize(img, img_shape)
    return img, label

def prepare_for_training(ds, batch_size=32, cache=True, shuffle_buffer_size=1000):
    '''
    TensorFlow Data API를 이용해 data batch를 만드는 함수
    '''
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    ds = ds.shuffle(buffer_size=shuffle_buffer_size)
    ds = ds.repeat()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return ds

    
def load_data(data_path, batch_size=32):
    '''
    데이터를 만들기 위해 필요한 함수들을 호출하고 데이터를 리턴해주는 함수
    '''
    class_names = [cls for cls in os.listdir(data_path) if cls != '.DS_Store']
    data_path = pathlib.Path(data_path)

    list_ds = tf.data.Dataset.list_files(str(data_path/'*/*'))
    labeled_ds = list_ds.map(lambda x: process_path(x, class_names, img_shape=(224, 224)))
    ds = prepare_for_training(labeled_ds, batch_size=batch_size)

    return ds

## Define Model

In [5]:
class Model(tf.keras.Model):
    def __init__(self, num_classes=10, freeze=False):
        super(Model, self).__init__()
        self.backbone = EfficientNetB0(include_top=False, input_shape=(224, 224, 3), weights='imagenet' if not freeze else None)
        if freeze:
            for layer in self.backbone.layers:
                layer.trainable = False
        self.global_avg_pooling = GlobalAveragePooling2D()
        self.classifier = Dense(num_classes, activation='softmax')

    def call(self, inputs, training=True):
        x = self.backbone(inputs, training=training)
        x = self.global_avg_pooling(x)
        return self.classifier(x)

## Define Trainer

In [7]:
class Trainer:
    def __init__(self, model, epochs, batch, loss_fn, optimizer, steps_per_epoch):
        self.model = model
        self.epochs = epochs
        self.batch = batch
        self.loss_fn = loss_fn
        self.optimizer = optimizer
        self.steps_per_epoch = steps_per_epoch

    def train(self, train_dataset, train_metric):
        for epoch in range(self.epochs):
            print("\nStart of epoch %d" % (epoch + 1,))
            train_metric.reset_states()

            for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
                if step >= self.steps_per_epoch:
                    break
                with tf.GradientTape() as tape:
                    logits = self.model(x_batch_train, training=True)
                    loss_value = self.loss_fn(y_batch_train, logits)
                grads = tape.gradient(loss_value, self.model.trainable_weights)
                self.optimizer.apply_gradients(zip(grads, self.model.trainable_weights))
                train_metric.update_state(y_batch_train, logits)

                if step % 200 == 0:
                    print("Training loss (for one batch) at step %d: %.4f" %
                          (step, float(loss_value)))
                    print("Seen so far: %s samples" % ((step + 1) * self.batch))

            train_acc = train_metric.result()
            print("Training acc over epoch: %.4f" % (float(train_acc),))

# Determine steps per epoch based on your dataset size
train_length = 10000  # replace with the actual size of your dataset
steps_per_epoch = train_length // batch

# Usage Example
train_path = "/aiffel/aiffel/model-fit/data/30vnfoods/Train"
epoch = 5
batch = 32

model = Model(num_classes=10)
dataset = load_data(data_path=train_path, batch_size=batch)
loss_function = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()

trainer = Trainer(model=model, epochs=epoch, batch=batch, loss_fn=loss_function, optimizer=optimizer, steps_per_epoch=steps_per_epoch)

trainer.train(train_dataset=dataset, train_metric=train_acc_metric)



Start of epoch 1
Training loss (for one batch) at step 0: 2.3098
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.7458
Seen so far: 6432 samples
Training acc over epoch: 0.8051

Start of epoch 2
Training loss (for one batch) at step 0: 0.1986
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.5218
Seen so far: 6432 samples
Training acc over epoch: 0.9094

Start of epoch 3
Training loss (for one batch) at step 0: 0.0809
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.0720
Seen so far: 6432 samples
Training acc over epoch: 0.9370

Start of epoch 4
Training loss (for one batch) at step 0: 0.0713
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.1981
Seen so far: 6432 samples
Training acc over epoch: 0.9488

Start of epoch 5
Training loss (for one batch) at step 0: 0.0735
Seen so far: 32 samples
Training loss (for one batch) at step 200: 0.1916
Seen so far: 6432 samples
Training acc over epoch: 0.9572


## Model Test

In [8]:
# 모델 테스트 코드
test_ds = load_data(data_path=test_path)

for step_train, (x_batch_train, y_batch_train) in enumerate(test_ds.take(10)):
    prediction = model(x_batch_train)
    print("{}/{}".format(np.array(tf.equal(tf.argmax(y_batch_train, axis=1), tf.argmax(prediction, axis=1))).sum(), tf.argmax(y_batch_train, axis=1).shape[0]))

29/32
27/32
32/32
28/32
29/32
28/32
28/32
29/32
28/32
28/32


## Retrospective

steps_per_epoch를 설정하지 않아 prepare_for_training에서 repeat()으로 인해 밤새 학습하였음에도 모델이 종료되지 않는 문제가 있었다.<br/>
분명 들었던 내용이었음에도 실제 구축하다보니 빼먹는 부분이 많았고 GPT가 없었다면 Custom을 완성하지 못했을 것 같다.<br/>
<br/>
backbone model은 Imagenet에서 뛰어난 성능을 보여주는 EfficientNet의 가장 기초 모델을 사용했지만 테스트할 때마다 보여주는 성능은 놀랍다.<br/><br/>
간편하게 모델을 불러오고 학습하는 과정을 한줄의 코드로 사용하는 것 보다 원리를 이해하려고 노력하다보니 낮설던 코드와 조금은 친해진 것 같다.